In [2]:
import pandas as pd
import numpy as np

In [5]:
input_dir = 'C:/ZhangLI/Codes/DataSet/MoA/'

In [75]:
train_features = pd.read_csv(input_dir+'train_features.csv')
train_targets_scored = pd.read_csv(input_dir+'train_targets_scored.csv')
train_targets_nonscored = pd.read_csv(input_dir+'train_targets_nonscored.csv')  # 评分的二进制MoA目标
test_features = pd.read_csv(input_dir+'test_features.csv')
sample_submission = pd.read_csv(input_dir+'sample_submission.csv')
train_drug = pd.read_csv(input_dir+'train_drug.csv')

In [13]:
train_features
train_targets_scored
train_targets_nonscored
train_features

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,trt_cp,24,D2,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,...,0.1969,0.0262,-0.8121,0.3434,0.5372,-0.3246,0.0631,0.9171,0.5258,0.4680
23810,id_fffb70c0c,trt_cp,24,D2,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,...,0.4286,0.4426,0.0423,-0.3195,-0.8086,-0.9798,-0.2084,-0.1224,-0.2715,0.3689
23811,id_fffc1c3f4,ctl_vehicle,48,D2,0.3942,0.3756,0.3109,-0.7389,0.5505,-0.0159,...,0.5409,0.3755,0.7343,0.2807,0.4116,0.6422,0.2256,0.7592,0.6656,0.3808
23812,id_fffcb9e7c,trt_cp,24,D1,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,...,-0.1105,0.4258,-0.2012,0.1506,1.5230,0.7101,0.1732,0.7015,-0.6290,0.0740


In [76]:
# 来评分  评个锤子的分
target_cols = train_targets_scored.drop('sig_id', axis=1).columns.values.tolist()
target_nonsc_cols = train_targets_nonscored.drop('sig_id', axis=1).columns.values.tolist()
nonctr_id = train_features.loc[train_features['cp_type']!='ctl_vehicle','sig_id'].tolist()
tmp_con1 = [i in nonctr_id for i in train_targets_scored['sig_id']]
#tmp_con1

In [77]:
sc_dic = {}
feat_dic = {}
mat_cor = pd.DataFrame(np.corrcoef(train_targets_scored.drop('sig_id',axis = 1)[tmp_con1].T,
                      train_targets_nonscored.drop('sig_id',axis = 1)[tmp_con1].T))
mat_cor2 = mat_cor.iloc[(train_targets_scored.shape[1]-1):,0:train_targets_scored.shape[1]-1]
mat_cor2.index = target_nonsc_cols
mat_cor2.columns = target_cols
mat_cor2 = mat_cor2.dropna()
mat_cor2_max = mat_cor2.abs().max(axis = 1)

q_n_cut = 0.9
target_nonsc_cols2 = mat_cor2_max[mat_cor2_max > np.quantile(mat_cor2_max,q_n_cut)].index.tolist()
print(len(target_nonsc_cols2))

GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]
feat_dic['gene'] = GENES
feat_dic['cell'] = CELLS

33


In [78]:
# sample norm 
q2 = train_features[feat_dic['gene']].apply(np.quantile,axis = 1,q = 0.25).copy()
q7 = train_features[feat_dic['gene']].apply(np.quantile,axis = 1,q = 0.75).copy()
qmean = (q2+q7)/2
train_features[feat_dic['gene']] = (train_features[feat_dic['gene']].T - qmean.values).T
q2 = test_features[feat_dic['gene']].apply(np.quantile,axis = 1,q = 0.25).copy()
q7 = test_features[feat_dic['gene']].apply(np.quantile,axis = 1,q = 0.75).copy()
qmean = (q2+q7)/2
test_features[feat_dic['gene']] = (test_features[feat_dic['gene']].T - qmean.values).T

q2 = train_features[feat_dic['cell']].apply(np.quantile,axis = 1,q = 0.25).copy()
q7 = train_features[feat_dic['cell']].apply(np.quantile,axis = 1,q = 0.72).copy()
qmean = (q2+q7)/2
train_features[feat_dic['cell']] = (train_features[feat_dic['cell']].T - qmean.values).T
qmean2 = train_features[feat_dic['cell']].abs().apply(np.quantile,axis = 1,q = 0.75).copy()+4
train_features[feat_dic['cell']] = (train_features[feat_dic['cell']].T / qmean2.values).T.copy()

q2 = test_features[feat_dic['cell']].apply(np.quantile,axis = 1,q = 0.25).copy()
q7 = test_features[feat_dic['cell']].apply(np.quantile,axis = 1,q = 0.72).copy()
qmean = (q2+q7)/2
test_features[feat_dic['cell']] = (test_features[feat_dic['cell']].T - qmean.values).T
qmean2 = test_features[feat_dic['cell']].abs().apply(np.quantile,axis = 1,q = 0.75).copy()+4
test_features[feat_dic['cell']] = (test_features[feat_dic['cell']].T / qmean2.values).T.copy()

In [79]:
# remove ctl
train = train_features.merge(train_targets_scored, on='sig_id')
train = train.merge(train_targets_nonscored[['sig_id']+target_nonsc_cols2], on='sig_id')
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)
target = train[['sig_id']+target_cols]
target_ns = train[['sig_id']+target_nonsc_cols2]

train0 = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

target_cols = target.drop('sig_id', axis=1).columns.values.tolist()


In [80]:
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)
target = train[['sig_id']+target_cols]
target_ns = train[['sig_id']+target_nonsc_cols2]

train0 = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

target_cols = target.drop('sig_id', axis=1).columns.values.tolist()


In [38]:
vc2

Index(['87d714366', '9f80f3f77', '8b87a7a83', '5628cb3ee', 'd08af5d4b',
       '292ab2c28', 'd50f18348', 'd1b47f29d'],
      dtype='object')

In [81]:
# ---
# drug ids
tar_sig = target['sig_id'].tolist()
train_drug = train_drug.loc[[i in tar_sig for i in train_drug['sig_id']]]
target = target.merge(train_drug, on='sig_id', how='left')

# LOCATE DRUGS
vc = train_drug.drug_id.value_counts()
vc1 = vc.loc[vc <= 19].index
vc2 = vc.loc[vc > 19].index



In [82]:
feature_cols = []
for key_i in feat_dic.keys():
    value_i = feat_dic[key_i]
    print(key_i,len(value_i))
    feature_cols += value_i
len(feature_cols)
from copy import deepcopy as dp
feature_cols0 = dp(feature_cols)
feature_cols0

oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))
oof
predictions.shape  # 分很多类 (3624, 206)
oof.shape  # (21948, 206)

gene 772
cell 100


(21948, 206)

In [89]:
import os
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import warnings
warnings.filterwarnings('ignore')

SEED = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
def seed_everything(seed=42):
    random.seed(seed)  # seed 确定 随机数确定
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)  # CPU设定种子
    torch.cuda.manual_seed(seed)  # GPU设定种子
    # 将这个 flag 置为True的话，每次返回的卷积算法将是确定的，即默认算法。
    # 如果配合上设置 Torch 的随机种子为固定值的话，应该可以保证每次运行网络的时候相同输入的输出是固定的
    torch.backends.cudnn.deterministic = True  
# for seed in SEED:
#     print(seed)
seed = 1
seed_everything(1)

# sig_id	cp_time	cp_dose	g-0	g-1	g-2	g-3	g-4	g-5	g-6	...	niemann-pick_c1-like_1_protein_antagonist	omega_3_fatty_acid_stimulant	quorum_sensing_signaling_modulator	reducing_agent	ror_inverse_agonist	sars_coronavirus_3c-like_protease_inhibitor	selective_estrogen_receptor_modulator_(serm)	sphingosine_1_phosphate_receptor_agonist	steryl_sulfatase_inhibitor	tyrosine_phosphatase_inhibitor
#  0	id_000644bb2	24	D1	1.051913	0.547612	-0.257988	-0.630888	-0.204488	-1.022088	-1.032088	...	0
folds = train0.copy()
feature_cols = dp(feature_cols0)
# sig_id	5-alpha_reductase_inhibitor	11-beta-hsd1_inhibitor	acat_inhibitor	acetylcholine_receptor_agonist	acetylcholine_receptor_antagonist	acetylcholinesterase_inhibitor	adenosine_receptor_agonist	adenosine_receptor_antagonist	adenylyl_cyclase_activator	...	tyrosine_kinase_inhibitor	ubiquitin_specific_protease_inhibitor	vegfr_inhibitor	vitamin_b	vitamin_d_receptor_agonist	wnt_inhibitor	drug_id_x	drug_id_y	drug_id_x	drug_id_y
#  0	id_000644bb2	0	0	0	
target2 = target.copy()

dct1 = {}; dct2 = {}
skf = MultilabelStratifiedKFold(n_splits = 5)

tmp = target2.groupby('drug_id')[target_cols].mean().loc[vc1]
tmp_idx = tmp.index.tolist()
tmp_idx.sort()

In [88]:
tmp

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
67c879e79,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
d488d031d,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52d1e6f43,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
83a9ea167,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30aa2f709,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b01cf573a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5bbc89d67,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fb8a93a83,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0e6ee26ff,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
tmp_idx

['00199ff52',
 '00251fc41',
 '00321ea80',
 '0034c0847',
 '0060e686f',
 '00898e82c',
 '009a5da05',
 '00d2de1d8',
 '00dba5599',
 '014d175d7',
 '015a48b37',
 '0166f205e',
 '01872549f',
 '018e6eb14',
 '018fbfcdb',
 '01af4c881',
 '01b66f497',
 '01ee58f64',
 '02084574f',
 '0212c63a7',
 '0222d944c',
 '022f3ae8e',
 '0233971e0',
 '0249e20c4',
 '025d91250',
 '026c86906',
 '02858eca8',
 '02ae89796',
 '02b14dcc0',
 '031afe482',
 '032128cc7',
 '03478f3f9',
 '035a6a762',
 '037888d25',
 '0393401cc',
 '03b09563c',
 '03b81e62e',
 '03b86deca',
 '0438952f6',
 '043fedc3f',
 '0472f88f4',
 '04765e596',
 '04791bbdf',
 '047d037a9',
 '04f5356dc',
 '04fdc0c19',
 '050116d56',
 '051a078dd',
 '053b1881a',
 '053bd4f4a',
 '053ebdb54',
 '0553c169f',
 '059eeaab7',
 '05e9ef087',
 '06008aff2',
 '062a92da6',
 '0643dfd76',
 '0672af23e',
 '06787fefe',
 '0691eefdd',
 '06af533b3',
 '06b61c0ea',
 '06bc90c08',
 '06c3860e2',
 '06cb74093',
 '070589dd4',
 '071754ad0',
 '071afe3da',
 '071fd1189',
 '07502fc59',
 '0760632c4',
 '07ab